In [4]:
import requests

import sys
sys.path.append('../')

import os

import zipfile

from tqdm import tqdm

import dask.dataframe as dd

import concurrent.futures

In [2]:
url = "https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD"
query_parameters = {"downloadformat": "csv"}

response = requests.get(url, params=query_parameters)

In [4]:
response.url, response.ok, response.status_code

('https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD?downloadformat=csv',
 True,
 200)

In [ ]:
fname = "../Data/25_3_data_getting/gdp.zip"
if not os.path.exists(fname):
    with open(fname, "wb") as file:
        file.write(response.content)
else:
    print("File already exists")

with zipfile.ZipFile(fname, "r") as zip_ref:
    zip_ref.extractall("../Data/25_3_data_getting")
    print("Files extracted")

File already exists
Files extracted


In [29]:
url = "https://databank.worldbank.org/data/download/WDI_CSV.zip"
response = requests.get(url, stream=True)
print(response.headers)

{'Date': 'Thu, 27 Mar 2025 19:11:58 GMT', 'Content-Type': 'application/octet-stream', 'Content-Length': '271860602', 'Connection': 'keep-alive', 'Last-Modified': 'Tue, 25 Mar 2025 17:30:13 GMT', 'ETag': '0x8DD6BC2B32A7F92', 'x-ms-request-id': '65444f76-601e-0006-0d4a-9f18d0000000', 'x-ms-version': '2009-09-19', 'x-ms-lease-status': 'unlocked', 'x-ms-blob-type': 'BlockBlob', 'x-azure-ref': '20250327T191158Z-165cd9497649x25vhC1CPH42as0000001kbg00000000f083', 'Cache-Control': 'public, max-age=3600', 'x-fd-int-roxy-purgeid': '83716666', 'X-Cache': 'TCP_HIT', 'X-Cache-Info': 'L1_T2', 'Accept-Ranges': 'bytes'}


In [ ]:
chunksize = 10*1024

with open("../Data/25_3_data_getting/WDI_CSV.zip", mode="wb") as file:
    for chunk in tqdm(response.iter_content(chunk_size=chunksize), total = int(response.headers['Content-Length']) / chunksize):
        file.write(chunk)

100%|█████████▉| 26518/26548.8869140625 [01:42<00:00, 274.52it/s]/Users/asger/.pyenv/versions/3.13.1/envs/base/lib/python3.13/site-packages/tqdm/std.py:636: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 26549/26548.8869140625 [01:42<00:00, 259.39it/s]


In [31]:
fname = "../Data/25_3_data_getting/WDI_CSV.zip"
with zipfile.ZipFile(fname, "r") as zip_ref:
    zip_ref.extractall("../Data/25_3_data_getting")
    print("Files extracted")

Files extracted


the below download took 17:34 on DSB network

In [34]:
data_home = "../Data/25_3_data_getting/"
fname = "aisdk.zip"
url = "https://web.ais.dk/aisdata/aisdk-2025-03-24.zip"
response = requests.get(url, stream=True)
print(response.headers)
print(response.ok)

chunksize = 10*1024
length = int(response.headers['Content-Length']) / chunksize
with open(data_home + fname, mode="wb") as file:
    for chunk in tqdm(response.iter_content(chunk_size=chunksize), total = length):
        file.write(chunk)

with zipfile.ZipFile(data_home + fname, "r") as zip_ref:
    zip_ref.extractall(data_home)
    print("Files extracted")

{'Date': 'Thu, 27 Mar 2025 19:24:37 GMT', 'Server': 'Apache', 'Last-Modified': 'Thu, 27 Mar 2025 05:16:36 GMT', 'ETag': '"2621de6e-6314c0ed13b89"', 'Accept-Ranges': 'bytes', 'Content-Length': '639753838', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'application/zip', 'Set-Cookie': 'ADC_SESSION=!Mo7fDA2mDskpRjWqZDt8cW8pGz3Zfrz6BUgZU+D5bdE3ob7zNTHLTfi/vDzFA2nO/5h5Bu5brHrRUhA=; path=/; Httponly; Secure'}
True


100%|█████████▉| 62464/62475.9607421875 [17:34<00:00, 139.33it/s] /Users/asger/.pyenv/versions/3.13.1/envs/base/lib/python3.13/site-packages/tqdm/std.py:636: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 62476/62475.9607421875 [17:34<00:00, 59.27it/s] 


Files extracted


Parallel download of the files seem to be significantly faster!!! 4 workers seem to be the safe choice for slow home connections

In [ ]:
url = "https://web.ais.dk/aisdata/aisdk-2025-03-24.zip"
data_home = "../Data/25_3_data_getting/faster/"
fname = os.path.join(data_home, url.split("/")[-1])

response = requests.head(url)
file_size = int(response.headers.get("Content-Length", 0))
num_chunks = 4  # Number of parallel chunks (adjust based on network speed)
chunk_size = file_size // num_chunks

print(f"File size: {file_size / 1e6:.2f} MB, Chunk size: {chunk_size / 1e6:.2f} MB")

# Function to download a specific chunk
def download_chunk(start, end, chunk_idx, progress_bar):
    headers = {"Range": f"bytes={start}-{end}"}
    chunk_fname = f"{fname}.part{chunk_idx}"
    
    with requests.get(url, headers=headers, stream=True) as r, open(chunk_fname, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024 * 1024):  # 1 MB chunks
            f.write(chunk)
            progress_bar.update(len(chunk))

    return chunk_fname

chunk_ranges = [(i * chunk_size, (i + 1) * chunk_size - 1, i) for i in range(num_chunks)]
chunk_ranges[-1] = (chunk_ranges[-1][0], file_size - 1, num_chunks - 1)  # Ensure last chunk gets remaining bytes

with tqdm(total=file_size, unit="B", unit_scale=True, desc="Downloading") as progress_bar:
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_chunks) as executor:
        chunk_files = list(executor.map(lambda args: download_chunk(*args, progress_bar), chunk_ranges))

with open(fname, "wb") as output_file:
    for chunk_file in chunk_files:
        with open(chunk_file, "rb") as part:
            output_file.write(part.read())
        os.remove(chunk_file)  # Delete temporary chunk files

print(f"Download complete: {fname}")

File size: 639.75 MB, Chunk size: 63.98 MB


Downloading:  95%|█████████▌| 609M/640M [03:45<00:34, 885kB/s]  